# Day 24

## Part One

<!-- Insert Markdown Here -->

---

In [18]:
# Initialise
with open('Day24.in') as f:
    inputs = [tuple(i[:-1].split(' ')) for i in f.readlines()]

In [19]:
inputs[0:5]

[('inp', 'w'),
 ('mul', 'x', '0'),
 ('add', 'x', 'z'),
 ('mod', 'x', '26'),
 ('div', 'z', '1')]

In [136]:
class ALU:
    def __init__(self, x=0, y=0, z=0, w=0):
        self.d = {
            'x': x,
            'y': y,
            'z': z,
            'w': w
        }
    
    def __getitem__(self, item):
        return self.d[item]
    
    def __setitem__(self, item, val):
        if item in self.d:
            self.d[item] = val
        else:
            raise KeyError(f"{item} not in (x, y, z, w).")
    
    def __contains__(self, var):
        return var in self.d
    
    def __iter__(self):
        return iter(self.d)
    
    def __str__(self):
        return "ALU" + str(self.d).replace("'","")
    
    def __repr__(self):
        return str(self)
    
    def inp(self, a, b):
        self.d[a] = int(b)
    
    def add(self, a, b):
        if b not in self:
            self.d[a] += int(b)
        else:
            self.d[a] += self.d[b]
    
    def mul(self, a, b):
        if b not in self:
            self.d[a] *= int(b)
        else:
            self.d[a] *= self.d[b]
    
    def div(self, a, b):
        if b not in self:
            self.d[a] //= int(b)
        else:
            self.d[a] //= self.d[b]
    
    def mod(self, a, b):
        if self.d[a] < 0:
            raise ZeroDivisionError(f"attempting to execute mod with {a}<0 ({self.d[a]}<0) will cause the program to crash and might even damage the ALU")
        if b not in self:
            self.d[a] %= int(b)
        else:
            self.d[a] %= self.d[b]
    
    def eql(self, a, b):
        if b not in self:
            self.d[a] = 1 if (self.d[a] == int(b)) else 0 
        else:
            self.d[a] = 1 if (self.d[a] == self.d[b]) else 0
    
    def parse(self, instruction, inp, verbose=False):
        if verbose:
            print(f"{self} <- {instruction} = ", end='\t')
        func, a, b, *_ = instruction + (None,)
        if func == 'inp':
            b = inp.pop(0)
        getattr(self, func)(a, b)
        print(self)
    
    def reset(self):
        for key in self:
            self[key] = 0
    
    def __call__(self, instructions, model, expected=14, reset=False, verbose=False):
        inputs = [int(i) for i in str(model)]
        og = inputs.copy()
        if (0 in inputs) or (len(inputs) != expected):
            raise ZeroDivisionError(f"{inputs} is not of expected length or contains a 0")
        
        if reset:
            rdict = self.d.copy()
        
        if verbose:
            last_inp = None
            
            vdict = self.d.copy()
        
        success = False
        i = 0
        
        for instruction in instructions:
            if instruction[0] == 'inp':
                i += 1
                if (i-1) and verbose:
                    print(f"Instruction set {i-1} with input {last_inp}={og[i-2]} results in {vdict} -> {self}")
                    vdict = self.d.copy()
                last_inp = instruction[1]
            if i > expected:
                break
            self.parse(instruction, inputs, verbose)
        
        if self['z'] == 0:
            success = True
        
        if reset:
            self.d = rdict
        
        return success
        

In [225]:
def MONAD(model, x=0, y=0, z=0, w=0):
    inputs = [int(i) for i in str(model)]
    funcs = [
        lambda i, x=0, y=0, z=0, w=0: (1, i+3, 26*z + i + 3, i),
        lambda i, x=0, y=0, z=0, w=0: (1, i+12, 26*z + i + 12, i),
        lambda i, x=0, y=0, z=0, w=0: (1, i+9, 26*z + i + 9, i),
        lambda i, x=0, y=0, z=0, w=0: ((t:=0 if (z%26) - 6 == i else 1), t*(i+12), (z//26)*(25*t + 1) + t*(i+12), i),
        lambda i, x=0, y=0, z=0, w=0: (1, i+2, 26*z + i + 2, i),
        lambda i, x=0, y=0, z=0, w=0: ((t:=0 if (z%26) - 8 == i else 1), t*(i+1), (z//26)*(25*t + 1) + t*(i+1), i),
        lambda i, x=0, y=0, z=0, w=0: ((t:=0 if (z%26) - 4 == i else 1), t*(i+1), (z//26)*(25*t + 1) + t*(i+1), i),
        lambda i, x=0, y=0, z=0, w=0: (1, i+13, 26*z + i + 13, i),
        lambda i, x=0, y=0, z=0, w=0: (1, i+1, 26*z + i + 1, i),
        lambda i, x=0, y=0, z=0, w=0: (1, i+6, 26*z + i + 6, i),
        lambda i, x=0, y=0, z=0, w=0: ((t:=0 if (z%26) - 11 == i else 1), t*(i+2), (z//26)*(25*t + 1) + t*(i+2), i),
        lambda i, x=0, y=0, z=0, w=0: ((t:=0 if (z%26) == i else 1), t*(i+11), (z//26)*(25*t + 1) + t*(i+11), i),
        lambda i, x=0, y=0, z=0, w=0: ((t:=0 if (z%26) - 8 == i else 1), t*(i+10), (z//26)*(25*t + 1) + t*(i+10), i),
        lambda i, x=0, y=0, z=0, w=0: ((t:=0 if (z%26) - 7 == i else 1), t*(i+3), (z//26)*(25*t + 1) + t*(i+3), i)
    ]
    
    for f,i in zip(funcs, inputs):
        x, y, z, w = f(i, x, y, z, w)
        print(i, z%26, z)
        
    return x, y, z, w


In [366]:
def MONAD2(model, x=0, y=0, z=0, w=0):
    inputs = [int(i) for i in str(model)]
    
    zinc = lambda k: (
        lambda i, x=0, y=0, z=0, w=0: (1, i+k, 26*z + i + k, i)
    )
    zdec = lambda d, k: (
        lambda i, x=0, y=0, z=0, w=0: ((t:=0 if (z%26) - d == i else 1), t*(i+k), (z//26)*(25*t + 1) + t*(i+k), i)
    )
    
    dks = [(None, 3), (None, 12), (None, 9), (6, 12), (None, 2), (8, 1), (4, 1), 
           (None, 13), (None, 1), (None, 6), (11, 2), (0, 11), (8, 10), (7, 3)]
    
    zis = []
    
    funcs = [
        zinc(3),       #  1 : 5
        zinc(12),      #  2 : 1
        zinc(9),       #  3 : 1
        zdec(6, 12),   #  4 : i3 + 3 = 4
        zinc(2),       #  5 : 7
        zdec(8, 1),    #  6 : i5 - 6 = 1
        zdec(4, 1),    #  7 : i2 + 8 = 9
        zinc(13),      #  8 : 1
        zinc(1),       #  9 : 1
        zinc(6),       # 10 : 6
        zdec(11, 2),   # 11 : i10 - 5 = 1
        zdec(0, 11),   # 12 : i9 + 1 = 2
        zdec(8, 10),   # 13 : i8 + 5 = 6
        zdec(7, 3),    # 14 : i1 - 4 = 1
    ]  # 51147191161261
    
    s = 0
    for j, (f,i, (d,k)) in enumerate(zip(funcs, inputs, dks)):
        #print(i, z%26, z)
        s += k+i
        s %= 26
        if d is not None:
            t = zis.pop()
            print(f"{j+1}:", z, i, (z%26) - d,((z%26) - d) == i, t, z%26, zis)
        else:
            zis.append(i+k)
            print(f"{j+1}:", z, i, i+k, z%26, zis)
            
        x, y, z, w = f(i, x, y, z, w)
    
    return (x, y, z, w)

In [367]:
MONAD2(51147191161261)

1: 0 5 8 0 [8]
2: 8 1 13 8 [8, 13]
3: 221 1 10 13 [8, 13, 10]
4: 5756 4 4 True 10 10 [8, 13]
5: 221 7 9 13 [8, 13, 9]
6: 5755 1 1 True 9 9 [8, 13]
7: 221 9 9 True 13 13 [8]
8: 8 1 14 8 [8, 14]
9: 222 1 2 14 [8, 14, 2]
10: 5774 6 12 2 [8, 14, 2, 12]
11: 150136 1 1 True 12 12 [8, 14, 2]
12: 5774 2 2 True 2 2 [8, 14]
13: 222 6 6 True 14 14 [8]
14: 8 1 1 True 8 8 []


(0, 0, 0, 1)

In [318]:
8384//26

322

In [320]:
322//26

12

In [217]:
218254 % 26

10

In [137]:
a = ALU()
a

ALU{x: 0, y: 0, z: 0, w: 0}

In [161]:
a

ALU{x: 1, y: 4, z: 3905417026, w: 1}

In [138]:

for i in range(int(100)-1, 10, -1):
    try:
        a = ALU(x=5)
        r = a(inputs, i, expected=2, verbose=True)
        
    except ZeroDivisionError:
        pass
    if r:
        print(r)
        break
    a.reset()
    

ALU{x: 5, y: 0, z: 0, w: 0} <- ('inp', 'w') = 	ALU{x: 5, y: 0, z: 0, w: 9}
ALU{x: 5, y: 0, z: 0, w: 9} <- ('mul', 'x', '0') = 	ALU{x: 0, y: 0, z: 0, w: 9}
ALU{x: 0, y: 0, z: 0, w: 9} <- ('add', 'x', 'z') = 	ALU{x: 0, y: 0, z: 0, w: 9}
ALU{x: 0, y: 0, z: 0, w: 9} <- ('mod', 'x', '26') = 	ALU{x: 0, y: 0, z: 0, w: 9}
ALU{x: 0, y: 0, z: 0, w: 9} <- ('div', 'z', '1') = 	ALU{x: 0, y: 0, z: 0, w: 9}
ALU{x: 0, y: 0, z: 0, w: 9} <- ('add', 'x', '13') = 	ALU{x: 13, y: 0, z: 0, w: 9}
ALU{x: 13, y: 0, z: 0, w: 9} <- ('eql', 'x', 'w') = 	ALU{x: 0, y: 0, z: 0, w: 9}
ALU{x: 0, y: 0, z: 0, w: 9} <- ('eql', 'x', '0') = 	ALU{x: 1, y: 0, z: 0, w: 9}
ALU{x: 1, y: 0, z: 0, w: 9} <- ('mul', 'y', '0') = 	ALU{x: 1, y: 0, z: 0, w: 9}
ALU{x: 1, y: 0, z: 0, w: 9} <- ('add', 'y', '25') = 	ALU{x: 1, y: 25, z: 0, w: 9}
ALU{x: 1, y: 25, z: 0, w: 9} <- ('mul', 'y', 'x') = 	ALU{x: 1, y: 25, z: 0, w: 9}
ALU{x: 1, y: 25, z: 0, w: 9} <- ('add', 'y', '1') = 	ALU{x: 1, y: 26, z: 0, w: 9}
ALU{x: 1, y: 26, z: 0, w: 9} <- ('m

In [335]:
a = ALU(x=0, y=0, z=0, w=0)
a(inputs, 41698999894945, expected=14, verbose=True)

ALU{x: 0, y: 0, z: 0, w: 0} <- ('inp', 'w') = 	ALU{x: 0, y: 0, z: 0, w: 4}
ALU{x: 0, y: 0, z: 0, w: 4} <- ('mul', 'x', '0') = 	ALU{x: 0, y: 0, z: 0, w: 4}
ALU{x: 0, y: 0, z: 0, w: 4} <- ('add', 'x', 'z') = 	ALU{x: 0, y: 0, z: 0, w: 4}
ALU{x: 0, y: 0, z: 0, w: 4} <- ('mod', 'x', '26') = 	ALU{x: 0, y: 0, z: 0, w: 4}
ALU{x: 0, y: 0, z: 0, w: 4} <- ('div', 'z', '1') = 	ALU{x: 0, y: 0, z: 0, w: 4}
ALU{x: 0, y: 0, z: 0, w: 4} <- ('add', 'x', '13') = 	ALU{x: 13, y: 0, z: 0, w: 4}
ALU{x: 13, y: 0, z: 0, w: 4} <- ('eql', 'x', 'w') = 	ALU{x: 0, y: 0, z: 0, w: 4}
ALU{x: 0, y: 0, z: 0, w: 4} <- ('eql', 'x', '0') = 	ALU{x: 1, y: 0, z: 0, w: 4}
ALU{x: 1, y: 0, z: 0, w: 4} <- ('mul', 'y', '0') = 	ALU{x: 1, y: 0, z: 0, w: 4}
ALU{x: 1, y: 0, z: 0, w: 4} <- ('add', 'y', '25') = 	ALU{x: 1, y: 25, z: 0, w: 4}
ALU{x: 1, y: 25, z: 0, w: 4} <- ('mul', 'y', 'x') = 	ALU{x: 1, y: 25, z: 0, w: 4}
ALU{x: 1, y: 25, z: 0, w: 4} <- ('add', 'y', '1') = 	ALU{x: 1, y: 26, z: 0, w: 4}
ALU{x: 1, y: 26, z: 0, w: 4} <- ('m

False

In [151]:
43698*26 + 7 + 2

1136157

---

## Part Two

<!-- Insert Markdown Here -->

---

In [34]:
l= [1,2,3]

l.pop(0)

1

In [35]:
l

[2, 3]

In [7]:
a = 5

a %= 0

ZeroDivisionError: integer division or modulo by zero

In [6]:
a

-1

---